In [18]:
import numpy as np
import matplotlib.pyplot as plt
from ase.units import kB

import matplotlib.pyplot as plt
import MDAnalysis as mda

import numpy as np
from MDAnalysis.analysis.base import Results
from MDAnalysis.analysis.rdf import InterRDF
from MDAnalysis.transformations.boxdimensions import set_dimensions


import ase.io
from tqdm.auto import tqdm
import pandas as pd

In [19]:
#keep mda version '2.2.0'
mda.__version__

'2.2.0'

In [20]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# requires to unzip the .zip archive with the trajectories.
# Have you unzipped the archive?

In [21]:
def CEA(obs, pot, T):
    #assuming obs is an array of shape (N_samples, N_Prop_dim)
    #and pot is an array of shape (N_samples, N_ensemble)
    # T is the temperature in K
    # returns N_ensemble weighted average obs

    beta = 1/(kB*T)
    w_obs = []
    obs_mean = np.mean(obs, axis=1)
    pot_mean = np.mean(pot, axis=1)

    #print(obs_mean.shape)
    #print(pot_mean.shape)

    for i in range(pot.shape[-1]):
        #print(np.mean(obs * (pot[:,i] - pot_mean),axis=1).shape)
        w_obs_i = obs_mean - beta * (np.mean(obs * (pot[:,i] - pot_mean),axis=1) - obs_mean * np.mean(pot[:,i] - pot_mean))
        w_obs.append(w_obs_i)

    return w_obs   

In [22]:
N_H2O = 64

identifier = f"../../Atomistic_experiments/trajectories_H2O/DFT_size_extensive_MD_300K/run_{N_H2O}"

#start = ase.io.read("../../data/trajectories_H2O/DFT_size_extensive_MD_300K/run_64/start_64.xyz",":")
start = ase.io.read(f"../../Atomistic_experiments/trajectories_H2O/DFT_size_extensive_MD_300K/start_{N_H2O}.xyz",":")

print(start)

universe_params = dict(format="Chemfiles", topology_format="XYZ", guess_bonds=True)

results = Results()
results.ml = Results()
#results.ml.u = mda.Universe("../../data/trajectories_H2O/DFT_size_extensive_MD_300K/run_64/BPNN-H2O-md.xc.xyz", **universe_params)
results.ml.u = mda.Universe(f"{identifier}/BPNN-H2O-md.xc.xyz", **universe_params)
results.ml.u.trajectory.add_transformations(
    set_dimensions(start[0].get_cell_lengths_and_angles())
)

STEP = 1
INT = int(1/STEP * 500)

print(INT)

results.ml.start = 0
results.ml.stop = -1

params = {"nbins": 200, "range": (0, 6)}

int_rdf_OO = []
int_rdf_HH = []

O = results.ml.u.select_atoms("name O*")
H = results.ml.u.select_atoms("name H*")
rdf_OO = InterRDF(O, O, **params, exclude_same="residue")
rdf_HH = InterRDF(H, H, **params, exclude_same="residue")

for ts in tqdm(range(0,len(results.ml.u.trajectory),STEP)):
    
    rdf_OO.run(start=ts, stop=ts+1,step=1)
    rdf_HH.run(start=ts, stop=ts+1,step=1)
    int_rdf_HH.append((rdf_HH.bins, rdf_HH.rdf))    
    int_rdf_OO.append((rdf_OO.bins, rdf_OO.rdf))

with open(f'{identifier}/BPNN-H2O-md.committee_pot_0', 'r') as file:
    lines = file.readlines()

dat = []
for line in lines[1::2]:
    tmp = []
    for s in line.split():
        tmp.append(float(s))
    dat.append(tmp)

dat = np.array(dat[::STEP])

E_mean = np.mean(dat, axis=1)
E_UQ = np.std(dat, axis=1)


int_rdfmat = np.hstack([x[1][:,np.newaxis] for x in int_rdf_HH])
w_obs = CEA(int_rdfmat[:,INT:], dat[INT:,:], 300)

mean_rdf = np.mean(w_obs,axis=0)
std_rdf = np.std(w_obs,axis=0)

np.save(f"CEA_{N_H2O}H2O_1ns_mean_HH.npy", mean_rdf)
np.save(f"CEA_{N_H2O}H2O_1ns_std_HH.npy", std_rdf)

int_rdfmat = np.hstack([x[1][:,np.newaxis] for x in int_rdf_OO])
w_obs = CEA(int_rdfmat[:,INT:], dat[INT:,:], 300)

mean_rdf = np.mean(w_obs,axis=0)
std_rdf = np.std(w_obs,axis=0)

np.save(f"CEA_{N_H2O}H2O_1ns_mean_OO.npy", mean_rdf)
np.save(f"CEA_{N_H2O}H2O_1ns_std_OO.npy", std_rdf)
np.save(f"gridRDF_{N_H2O}H2O_1ns.npy", int_rdf_OO[0][0])


[Atoms(symbols='H128O64', pbc=True, cell=[12.417201000000016, 12.417201000000016, 12.417201000000016], atomtypes=..., bfactor=..., forces=..., residuenames=..., residuenumbers=..., calculator=SinglePointCalculator(...))]
500


100%|██████████| 20001/20001 [48:59<00:00,  6.80it/s] 


In [23]:
N_H2O = 256

identifier = f"../../Atomistic_experiments/trajectories_H2O/DFT_size_extensive_MD_300K/run_{N_H2O}"

#start = ase.io.read("../../data/trajectories_H2O/DFT_size_extensive_MD_300K/run_64/start_64.xyz",":")
try:
    start = ase.io.read(f"{identifier}/start.xyz",":")
except:
    start = ase.io.read(f"{identifier}/start_{N_H2O}.xyz",":")


universe_params = dict(format="CHEMFILES", topology_format="XYZ", guess_bonds=True)

results = Results()
results.ml = Results()
#results.ml.u = mda.Universe("../../data/trajectories_H2O/DFT_size_extensive_MD_300K/run_64/BPNN-H2O-md.xc.xyz", **universe_params)
results.ml.u = mda.Universe(f"{identifier}/BPNN-H2O-md.xc.xyz", **universe_params)
results.ml.u.trajectory.add_transformations(
    set_dimensions(start[0].get_cell_lengths_and_angles())
)
STEP = 1
INT = int(1/STEP * 500)

print(INT)

results.ml.start = 0
results.ml.stop = -1

params = {"nbins": 200, "range": (0, 6)}

int_rdf_OO = []
int_rdf_HH = []

O = results.ml.u.select_atoms("name O*")
H = results.ml.u.select_atoms("name H*")
rdf_OO = InterRDF(O, O, **params, exclude_same="residue")
rdf_HH = InterRDF(H, H, **params, exclude_same="residue")

for ts in tqdm(range(0,len(results.ml.u.trajectory),STEP)):
    
    rdf_OO.run(start=ts, stop=ts+1,step=1)
    rdf_HH.run(start=ts, stop=ts+1,step=1)
    int_rdf_HH.append((rdf_HH.bins, rdf_HH.rdf))    
    int_rdf_OO.append((rdf_OO.bins, rdf_OO.rdf))

with open(f'{identifier}/BPNN-H2O-md.committee_pot_0', 'r') as file:
    lines = file.readlines()

dat = []
for line in lines[1::2]:
    tmp = []
    for s in line.split():
        tmp.append(float(s))
    dat.append(tmp)

dat = np.array(dat[::STEP])

E_mean = np.mean(dat, axis=1)
E_UQ = np.std(dat, axis=1)


int_rdfmat = np.hstack([x[1][:,np.newaxis] for x in int_rdf_HH])
w_obs = CEA(int_rdfmat[:,INT:], dat[INT:,:], 300)

mean_rdf = np.mean(w_obs,axis=0)
std_rdf = np.std(w_obs,axis=0)

np.save(f"CEA_{N_H2O}H2O_1ns_mean_HH.npy", mean_rdf)
np.save(f"CEA_{N_H2O}H2O_1ns_std_HH.npy", std_rdf)
np.save(f"CEA_{N_H2O}int_rdf_HH.npy", int_rdfmat)

int_rdfmat = np.hstack([x[1][:,np.newaxis] for x in int_rdf_OO])
w_obs = CEA(int_rdfmat[:,INT:], dat[INT:,:], 300)

mean_rdf = np.mean(w_obs,axis=0)
std_rdf = np.std(w_obs,axis=0)

np.save(f"CEA_{N_H2O}H2O_1ns_mean_OO.npy", mean_rdf)
np.save(f"CEA_{N_H2O}H2O_1ns_std_OO.npy", std_rdf)
np.save(f"gridRDF_{N_H2O}H2O_1ns.npy", int_rdf_OO[0][0])
np.save(f"CEA_{N_H2O}int_rdf_OO.npy", int_rdfmat)


500


100%|██████████| 6424/6424 [19:50<00:00,  5.40it/s]
